In [2]:
# Installer bibliothèques
!pip install -q pyspark==3.5.1 findspark datasets sentence-transformers transformers \
                 scikit-learn xgboost nltk matplotlib seaborn umap-learn tqdm \
                 tensorflow torch pyspellchecker

^C


In [ ]:
!pip install

In [3]:
import datasets
print(datasets.__version__)

c:\Users\0000\Desktop\Cours_LLM\notebooks\pytorch_gpu\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


3.6.0


In [4]:
# Exécuter en cellule "Code" (bash) dans Colab

# NLTK ressources

import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
print("NLTK ready")



ModuleNotFoundError: No module named 'nltk'

In [8]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, explode, array, struct, avg, count, size, length
from pyspark.sql.types import StringType, FloatType, IntegerType, ArrayType, StructType, StructField
import pyspark.sql.functions as F

spark = SparkSession.builder \
    .appName("AI_Detection_SBERT_LSTM") \
    .config("spark.driver.memory", "12g") \
    .config("spark.executor.memory", "6g") \
    .config("spark.sql.shuffle.partitions", "16") \
    .config("spark.driver.maxResultSize", "4g") \
    .getOrCreate()

print(f"Spark version: {spark.version}")

Spark version: 3.5.1


In [9]:
from datasets import load_dataset
import pandas as pd

# Charger le dataset
ds = load_dataset("yaful/MAGE")
print(ds)

# Convertir en Pandas puis en Spark DataFrame
train_pd = pd.DataFrame(ds["train"][:])
valid_pd = pd.DataFrame(ds["validation"][:])
test_pd = pd.DataFrame(ds["test"][:])

# Créer des Spark DataFrames
train_df = spark.createDataFrame(train_pd)
valid_df = spark.createDataFrame(valid_pd)
test_df = spark.createDataFrame(test_pd)

print(f"\nTrain rows: {train_df.count()}")
train_df.show(3, truncate=50)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.csv:   0%|          | 0.00/404M [00:00<?, ?B/s]

valid.csv:   0%|          | 0.00/72.3M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/71.7M [00:00<?, ?B/s]

...

test_ood_set_gpt_para.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/319071 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/56792 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/60743 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'src'],
        num_rows: 319071
    })
    validation: Dataset({
        features: ['text', 'label', 'src'],
        num_rows: 56792
    })
    test: Dataset({
        features: ['text', 'label', 'src'],
        num_rows: 60743
    })
})

Train rows: 319071
+--------------------------------------------------+-----+---------+
|                                              text|label|      src|
+--------------------------------------------------+-----+---------+
|White girls very rarely date Asian men. Even in...|    1|cmv_human|
|I am a 23 year old male Indian American male. I...|    1|cmv_human|
|Take three people, Persons A, B, and C. They li...|    1|cmv_human|
+--------------------------------------------------+-----+---------+
only showing top 3 rows



In [10]:
import unicodedata
import re
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def clean_text_light(text):
    """Nettoyage de texte léger"""
    if text is None:
        return ""
    text = unicodedata.normalize("NFKC", text)
    text = text.lower()
    text = re.sub(r'\S+@\S+', ' ', text)
    text = re.sub(r'http\S+|www\.\S+', ' ', text)
    text = re.sub(r'[@#]\w+', ' ', text)
    text = re.sub(r'[^0-9a-z\s\.\,\!\?\:\;\-\'\[\]]+', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Enregistrer l'UDF
clean_text_udf = udf(clean_text_light, StringType())

# Appliquer le nettoyage
train_df = train_df.withColumn("text_clean", clean_text_udf(col("text")))
valid_df = valid_df.withColumn("text_clean", clean_text_udf(col("text")))
test_df = test_df.withColumn("text_clean", clean_text_udf(col("text")))

train_df.select("text_clean").show(3, truncate=100)

+----------------------------------------------------------------------------------------------------+
|                                                                                          text_clean|
+----------------------------------------------------------------------------------------------------+
|white girls very rarely date asian men. even in asia white men are much more popular than asian m...|
|i am a 23 year old male indian american male. i have never held hands with a girl, been kissed, o...|
|take three people, persons a, b, and c. they live in the us. person a was born here and is a citi...|
+----------------------------------------------------------------------------------------------------+
only showing top 3 rows



In [11]:
from collections import Counter
from spellchecker import SpellChecker
import numpy as np

spell = SpellChecker(language='en')

def extract_features_dict(text):
    """Extrait des features linguistiques avancées"""
    text_clean = clean_text_light(text)
    sents = sent_tokenize(text_clean)
    words = word_tokenize(text_clean)
    words_alpha = [w for w in words if w.isalpha()]
    n_words = len(words_alpha)
    n_sents = max(1, len(sents))

    ttr = len(set(words_alpha)) / n_words if n_words > 0 else 0.0
    avg_word_len = np.mean([len(w) for w in words_alpha]) if n_words > 0 else 0.0
    avg_sent_len = n_words / n_sents
    long_sents = sum(1 for s in sents if len(word_tokenize(s)) >= 20) / n_sents

    ctr = Counter(words_alpha)
    rep_rate = max(ctr.values()) / n_words if n_words > 0 else 0.0

    misspell = spell.unknown(words_alpha)
    spell_err_rate = len(list(misspell)) / n_words if n_words > 0 else 0.0

    return {
        "n_words": float(n_words),
        "n_sents": float(n_sents),
        "ttr": float(ttr),
        "avg_word_len": float(avg_word_len),
        "avg_sent_len": float(avg_sent_len),
        "long_sents_ratio": float(long_sents),
        "rep_rate": float(rep_rate),
        "spell_err_rate": float(spell_err_rate)
    }

# Définir le schema de retour
features_schema = StructType([
    StructField("n_words", FloatType()),
    StructField("n_sents", FloatType()),
    StructField("ttr", FloatType()),
    StructField("avg_word_len", FloatType()),
    StructField("avg_sent_len", FloatType()),
    StructField("long_sents_ratio", FloatType()),
    StructField("rep_rate", FloatType()),
    StructField("spell_err_rate", FloatType())
])

extract_features_udf = udf(extract_features_dict, features_schema)

# Appliquer l'extraction de features
train_df = train_df.withColumn("features", extract_features_udf(col("text_clean")))
train_df = train_df.select("*", "features.*")

train_df.select("text_clean", "n_words", "ttr", "avg_word_len").show(5)

+--------------------+-------+----------+------------+
|          text_clean|n_words|       ttr|avg_word_len|
+--------------------+-------+----------+------------+
|white girls very ...|  219.0| 0.6666667|   4.3607306|
|i am a 23 year ol...|  259.0| 0.6061776|    4.050193|
|take three people...|  298.0|0.54362416|   4.2114096|
|a work part-time ...|  397.0| 0.5188917|    4.299748|
|when police intro...|  213.0|  0.600939|   4.6995306|
+--------------------+-------+----------+------------+
only showing top 5 rows



In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Charger le modèle SBERT
model = SentenceTransformer('all-MiniLM-L6-v2')
EMB_DIM = model.get_sentence_embedding_dimension()
MAX_SENTS = 10

print(f"Embedding dimension: {EMB_DIM}")

# Broadcaster le modèle pour l'utiliser dans les workers
broadcast_model = spark.sparkContext.broadcast(model)

def compute_sent_embeddings(text, max_sents=MAX_SENTS):
    """Calcule les embeddings par phrase avec padding"""
    model = broadcast_model.value
    sents = sent_tokenize(text)[:max_sents]

    if len(sents) == 0:
        return [0.0] * (max_sents * EMB_DIM)

    emb = model.encode(sents, show_progress_bar=False)
    padded = np.zeros((max_sents, EMB_DIM))

    for i, e in enumerate(emb):
        padded[i] = e

    return padded.flatten().tolist()

def compute_mean_embedding(text):
    """Calcule l'embedding moyen du document"""
    model = broadcast_model.value
    sents = sent_tokenize(text)

    if len(sents) == 0:
        return [0.0] * EMB_DIM

    emb = model.encode(sents, show_progress_bar=False)
    return np.mean(emb, axis=0).tolist()

# Enregistrer les UDFs
sent_emb_udf = udf(compute_sent_embeddings, ArrayType(FloatType()))
mean_emb_udf = udf(compute_mean_embedding, ArrayType(FloatType()))

# ATTENTION: Pour gros datasets, traiter par batch avec mapPartitions
print("Génération des embeddings (peut prendre du temps)...")

# Option 1: Échantillonnage pour test rapide
SAMPLE_FRACTION = 0.1  # Utiliser 10% pour test rapide
train_sample = train_df.sample(fraction=SAMPLE_FRACTION, seed=42)

# Générer les embeddings
train_sample = train_sample.withColumn("sent_embeddings", sent_emb_udf(col("text_clean")))
train_sample = train_sample.withColumn("mean_embedding", mean_emb_udf(col("text_clean")))

# Cache pour réutilisation
train_sample.cache()
print(f"Embeddings générés pour {train_sample.count()} documents")

train_sample.select("text_clean", "sent_embeddings").show(2, truncate=50)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding dimension: 384
Génération des embeddings (peut prendre du temps)...


In [ ]:
from sklearn.model_selection import train_test_split

# Collecter les données (ATTENTION: seulement si ça tient en mémoire)
print("Collection des données pour training...")
train_collected = train_sample.select(
    "label",
    "sent_embeddings",
    "mean_embedding",
    "n_words", "n_sents", "ttr", "avg_word_len",
    "avg_sent_len", "long_sents_ratio", "rep_rate", "spell_err_rate"
).collect()

# Conversion en numpy arrays
y = np.array([row['label'] for row in train_collected])
sent_embs = np.array([row['sent_embeddings'] for row in train_collected]).reshape(-1, MAX_SENTS, EMB_DIM)
mean_embs = np.array([row['mean_embedding'] for row in train_collected])

print(f"Shapes - y: {y.shape}, sent_embs: {sent_embs.shape}, mean_embs: {mean_embs.shape}")

# Split stratifié
idx_train, idx_val = train_test_split(
    range(len(y)),
    test_size=0.15,
    random_state=42,
    stratify=y
)

X_seq_train = sent_embs[idx_train]
X_seq_val = sent_embs[idx_val]
X_mean_train = mean_embs[idx_train]
X_mean_val = mean_embs[idx_val]
y_train = y[idx_train]
y_val = y[idx_val]

print(f"Train: {len(y_train)}, Val: {len(y_val)}")
print(f"Distribution train: {np.bincount(y_train)}")

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Masking, LSTM, Bidirectional, Dense, Dropout, GlobalMaxPool1D
from tensorflow.keras.models import Model

tf.random.set_seed(42)

def build_lstm_model(max_sents=MAX_SENTS, emb_dim=EMB_DIM):
    """Modèle LSTM bidirectionnel avec masking"""
    inp = Input(shape=(max_sents, emb_dim), name='sent_embeddings')

    # Masking pour ignorer le padding
    x = Masking(mask_value=0.0)(inp)

    # Couches LSTM bidirectionnelles
    x = Bidirectional(LSTM(128, return_sequences=True, dropout=0.2))(x)
    x = Bidirectional(LSTM(64, return_sequences=True, dropout=0.2))(x)

    # Agrégation
    x = GlobalMaxPool1D()(x)

    # Couches denses
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.4)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.3)(x)

    # Output
    out = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=inp, outputs=out)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
    )
    return model

lstm_model = build_lstm_model()
lstm_model.summary()

In [ ]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        "lstm_sbert_best.h5",
        save_best_only=True,
        monitor="val_loss"
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-6
    )
]

print("\nDébut de l'entraînement...")
history = lstm_model.fit(
    X_seq_train, y_train,
    validation_data=(X_seq_val, y_val),
    epochs=200,
    batch_size=32,
    callbacks=callbacks,
       verbose=1
)


In [ ]:
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score,
    recall_score, confusion_matrix, classification_report
)
import matplotlib.pyplot as plt
import seaborn as sns

# Prédictions
y_val_prob = lstm_model.predict(X_seq_val).ravel()
y_val_pred = (y_val_prob >= 0.5).astype(int)

# Métriques
print("\n" + "="*60)
print("RÉSULTATS FINAUX")
print("="*60)
print(f"Accuracy:  {accuracy_score(y_val, y_val_pred):.4f}")
print(f"Precision: {precision_score(y_val, y_val_pred):.4f}")
print(f"Recall:    {recall_score(y_val, y_val_pred):.4f}")
print(f"F1-Score:  {f1_score(y_val, y_val_pred):.4f}")
print("\nClassification Report:")
print(classification_report(y_val, y_val_pred, target_names=['Human', 'AI']))

# Matrice de confusion
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Confusion matrix
cm = confusion_matrix(y_val, y_val_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0])
axes[0].set_title('Confusion Matrix')
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Actual')

# Courbes d'apprentissage
axes[1].plot(history.history['loss'], label='Train Loss')
axes[1].plot(history.history['val_loss'], label='Val Loss')
axes[1].set_title('Learning Curves')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
import umap

SAMPLE_SIZE = min(2000, X_mean_val.shape[0])
X_sample = X_mean_val[:SAMPLE_SIZE]
y_sample = y_val[:SAMPLE_SIZE]

print(f"\nGénération UMAP sur {SAMPLE_SIZE} échantillons...")
reducer = umap.UMAP(n_components=2, random_state=42, n_neighbors=15)
X_umap = reducer.fit_transform(X_sample)

plt.figure(figsize=(10, 8))
scatter = plt.scatter(
    X_umap[:, 0], X_umap[:, 1],
    c=y_sample,
    s=20,
    cmap='RdYlBu',
    alpha=0.6,
    edgecolors='black',
    linewidth=0.5
)
plt.colorbar(scatter, label='Label (0=Human, 1=AI)')
plt.title('UMAP Projection - Mean SBERT Embeddings', fontsize=14, fontweight='bold')
plt.xlabel('UMAP 1')
plt.ylabel('UMAP 2')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Define the base path for saving to Google Drive
drive_path = '/content/drive/MyDrive/IA_Detection_SBERT_LSTM'

# Create the directory if it doesn't exist
import os
os.makedirs(drive_path, exist_ok=True)

# Sauvegarder le modèle sur Google Drive
lstm_model.save(f"{drive_path}/final_lstm_sbert_model.h5")
print(f"\nModèle sauvegardé: {drive_path}/final_lstm_sbert_model.h5")

# Sauvegarder les embeddings en Parquet sur Google Drive pour réutilisation
# Note: This operation might still encounter Spark-related issues if the Spark session state is unstable.
train_sample.select("text", "label", "mean_embedding", "sent_embeddings") \
    .write.mode("overwrite") \
    .parquet(f"{drive_path}/embeddings_train.parquet")
print(f"Embeddings sauvegardés en Parquet: {drive_path}/embeddings_train.parquet")


print("\n" + "="*60)
print("PIPELINE TERMINÉ!")
print("="*60)


In [ ]:
# Sauvegarder le modèle
lstm_model.save("final_lstm_sbert_model.h5")
print("\nModèle sauvegardé: final_lstm_sbert_model.h5")

# Optionnel: Sauvegarder les embeddings en Parquet pour réutilisation

train_sample.select("text", "label", "mean_embedding", "sent_embeddings") \
    .write.mode("overwrite") \
    .parquet("/content/embeddings_train.parquet")
print("Embeddings sauvegardés en Parquet")


print("\n" + "="*60)
print("PIPELINE TERMINÉ!")
print("="*60)



In [ ]:
# Arrêter Spark
spark.stop()